In [1]:
# header files
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

In [2]:
# number of workers
num_workers = 0

# batch size
batch_size = 20

# percentage validation data
valid_size = 0.2

# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# load dataset for CIFAR-10
train_data = datasets.CIFAR10('data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False, download=True, transform=transform)

# shuffle the indices to be used for training and validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define training and validation samplers
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, sampler = train_sampler, num_workers = num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, sampler = valid_sampler, num_workers = num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers = num_workers)

# classes for classification
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# view image
import cv2

dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

image = images[0][0]
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
# define CNN
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # input image of size (32 x 32 x 3)
        self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
        
        # cnn1 layer of size (16 x 16 x 16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        
        # cnn2 layer of size (8 x 8 x 32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding = 1)
        
        # max-pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # fc1 layer
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        
        # fc2 layer
        self.fc2 = nn.Linear(500, 10)
        
        # dropout layer
        self.dropout = nn.Dropout(0.25)
        
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [5]:
# define loss and optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.01)

In [8]:
num_epochs = 10

# training
for epoch in range(0, num_epochs):  # loop over the dataset multiple times

    training_loss = 0.0
    valid_loss = 0.0
    
    net.train()
    for data, label in train_loader:
        optimizer.zero_grad()
        
        pred_output = net(data)
        loss = criterion(pred_output, label)
        
        loss.backward()
        optimizer.step()
        training_loss = training_loss + (loss.item() * data.size(0))
    
    net.eval()
    for data, label in valid_loader:
        pred_output = net(data)
        loss = criterion(pred_output, label)
        valid_loss = valid_loss + (loss.item() * data.size(0))
    
    training_loss = training_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    
    print("Training loss: " + str(training_loss) + " and Valid Loss: " + str(valid_loss) + "\n\n")
print('Finished Training')

Training loss: 1.50866180596 and Valid Loss: 1.42422300816


Training loss: 1.3841996417 and Valid Loss: 1.30255975866


Training loss: 1.28053286946 and Valid Loss: 1.20385746217


Training loss: 1.19811038217 and Valid Loss: 1.10849313873


Training loss: 1.11865341745 and Valid Loss: 1.04799854648


Training loss: 1.05435467091 and Valid Loss: 1.02154697371


Training loss: 0.999149248317 and Valid Loss: 0.959700778127


Training loss: 0.948836392298 and Valid Loss: 0.929198274314


Training loss: 0.903846905023 and Valid Loss: 0.903964693844


Training loss: 0.863266421154 and Valid Loss: 0.866632081628


Finished Training


In [12]:
# testing phase
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

net.eval()
for data, label in test_loader:
    pred_output = net(data)
    loss = criterion(pred_output, label)
    test_loss = test_loss + (loss.item() * data.size(0))
    
    _, pred = torch.max(pred_output, 1)
    correct_tensor = pred.eq(label.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())
    
    for i in range(0, batch_size):
        class_correct[label.data[i]] = class_correct[label.data[i]] + correct[i].item()
        class_total[label.data[i]] = class_total[label.data[i]] + 1
    
test_loss = test_loss / len(test_loader.dataset)
print("Test Loss: " + str(test_loss) + "\n")
print("Testing Accuracy: " + str(100.0 * (np.sum(class_correct) / np.sum(class_total))))

Test Loss: 0.86321675092

Testing Accuracy: 69.69
